# Agenda

1. The iterator protocol
    - Adding iteration to your classes
    - Different techniques for that
2. Generator functions
3. Generator comprehensions
4. Decorators
5. Threading and multiprocessing 

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
        
p1 = Person('name1')        
p2 = Person('name2')

print(p1.greet())
print(p2.greet())

Hello, name1!
Hello, name2!


In [2]:
import weakref

In [4]:
p3 = weakref.ref(p2)


In [5]:
p3

<weakref at 0x1106405e0; to 'Person' at 0x11063f460>

In [7]:
p3.ref

AttributeError: 'weakref' object has no attribute 'ref'